# GPU Testing for CNN

In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using plaidml.keras.backend backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

INFO:plaidml:Opening device "metal_amd_radeon_pro_555x.0"


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
 6784/60000 [==>...........................] - ETA: 28s - loss: 0.9556 - acc: 0.6879

/Users/karan/anaconda3/lib/python3.7/site-packages/numpy/ctypeslib.py:436: RuntimeWarning: Invalid PEP 3118 format string: '&<f'
  return array(obj, copy=False)


60000/60000 [==============================] - 25s 422us/step - loss: 0.2639 - acc: 0.9189 - val_loss: 0.0530 - val_acc: 0.9834
Epoch 2/12
60000/60000 [==============================] - 22s 362us/step - loss: 0.0868 - acc: 0.9743 - val_loss: 0.0399 - val_acc: 0.9872
Epoch 3/12
60000/60000 [==============================] - 22s 361us/step - loss: 0.0650 - acc: 0.9808 - val_loss: 0.0340 - val_acc: 0.9891
Epoch 4/12
60000/60000 [==============================] - 22s 361us/step - loss: 0.0547 - acc: 0.9831 - val_loss: 0.0300 - val_acc: 0.9898
Epoch 5/12
60000/60000 [==============================] - 22s 360us/step - loss: 0.0484 - acc: 0.9858 - val_loss: 0.0293 - val_acc: 0.9908
Epoch 6/12
60000/60000 [==============================] - 22s 362us/step - loss: 0.0428 - acc: 0.9867 - val_loss: 0.0278 - val_acc: 0.9907
Epoch 7/12
60000/60000 [==============================] - 22s 359us/step - loss: 0.0374 - acc: 0.9887 - val_loss: 0.0313 - val_acc: 0.9904
Epoch 8/12
60000/60000 [==============

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score

pred = model.predict(x_test, batch_size=512, verbose=1)
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

10000/10000 [==============================] - 2s 196us/step
For threshold:  0.1
Micro-average quality numbers
Precision: 0.9807, Recall: 0.9970, F1-measure: 0.9888
For threshold:  0.2
Micro-average quality numbers
Precision: 0.9865, Recall: 0.9955, F1-measure: 0.9910
For threshold:  0.3
Micro-average quality numbers
Precision: 0.9898, Recall: 0.9947, F1-measure: 0.9923
For threshold:  0.4
Micro-average quality numbers
Precision: 0.9915, Recall: 0.9936, F1-measure: 0.9926
For threshold:  0.5
Micro-average quality numbers
Precision: 0.9932, Recall: 0.9924, F1-measure: 0.9928
For threshold:  0.6
Micro-average quality numbers
Precision: 0.9943, Recall: 0.9903, F1-measure: 0.9923
For threshold:  0.7
Micro-average quality numbers
Precision: 0.9952, Recall: 0.9891, F1-measure: 0.9921
For threshold:  0.8
Micro-average quality numbers
Precision: 0.9964, Recall: 0.9859, F1-measure: 0.9911
For threshold:  0.9
Micro-average quality numbers
Precision: 0.9977, Recall: 0.9811, F1-measure: 0.9893
